In [ ]:
%pip install requests
%pip install bs4

In [ ]:
import requests
from bs4 import BeautifulSoup


url = "https://en.wikipedia.org/wiki/World_War_II"

response = requests.get(url)

if response.status_code == 200:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract text data from the webpage (customize this based on the webpage's structure)
    webpage_text = " ".join([p.get_text() for p in soup.find_all("p")])
else:
    print("Failed to fetch webpage content")


print(webpage_text)


In [ ]:

%pip install transformers


In [ ]:
with open("webpage_data.txt", "w", encoding="utf-8") as file:
    file.write(webpage_text)

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
# Sample input text
# input_text = "Hello, this is an example sentence with some stop words and punctuation!"

# Remove HTML tags (if applicable)
# Remove punctuation, convert to lowercase, and tokenize
# input_text = BeautifulSoup(input_text, 'html.parser').get_text()
# input_text = webpage_text.lower()
# input_text = re.sub(f"[{re.escape(string.punctuation)}]", "", input_text)
# tokens = word_tokenize(input_text)

# # Remove stop words
# stop_words = set(stopwords.words('english'))
# filtered_tokens = [word for word in tokens if word not in stop_words]

# # Stemming (optional)
# stemmer = PorterStemmer()
# stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

# # Join tokens back into a sentence or use as needed
# processed_text = ' '.join(stemmed_tokens)
# print(processed_text)


In [ ]:
import nltk.data
nltk.download('punkt')

# Define the input and output file paths
input_file_path = "webpage_data.txt"  # Replace with your input file path
output_file_path = "tokenized_webpage_data.txt"  # Replace with your output file path

# Initialize the sentence tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Read the input text from the file
with open(input_file_path, 'r', encoding='utf-8') as input_file:
    input_text = input_file.read()

input_text = input_text.lower()
input_text = re.sub(f"[{re.escape(string.punctuation)}]", "", input_text)
tokens = word_tokenize(input_text)

# Remove stop words
stop_words = set(stopwords.words('english'))
filtered_tokens = [word for word in tokens if word not in stop_words]

# Stemming (optional)
# stemmer = PorterStemmer()
# stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

# Join tokens back into a sentence or use as needed
processed_text = ' '.join(filtered_tokens)
# print(processed_text)
input_text=processed_text

# Tokenize the input text into sentences
sentences = tokenizer.tokenize(input_text)

# Write each sentence to the output file on a new line
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    for sentence in sentences:
        output_file.write(sentence.strip() + '\n')

# import re
# re.sub(r'[^a-zA-Z0-9\s]', '', output_file_path) #Removing special characters

print(f"Sentences tokenized and saved to '{output_file_path}'.")


In [ ]:
%pip install transformers[torch,accelerate]>=0.20.1


In [ ]:
dict_faq="dict_faq.txt"


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

torch.utils.bottleneck = None

model_name = "gpt2"  # You can choose a different model if needed
model = GPT2LMHeadModel.from_pretrained(model_name).to('cuda:0')  # Move to GPU
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# from transformers import RobertaForCausalLM, RobertaTokenizer

# # Load the RoBERTa model and tokenizer
# model_name = "roberta"
# model = RobertaForCausalLM.from_pretrained("roberta").to('cuda:0')  # Move to GPU
# tokenizer = RobertaTokenizer.from_pretrained("roberta")


# Collect and preprocess your data (replace with actual data collection and preprocessing)
data = "tokenized_webpage_data.txt"

# Tokenize and format the data for training
train_dataset = TextDataset(tokenizer=tokenizer, file_path=data, block_size=128)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./fine-tuned-model1",
    overwrite_output_dir=True,
    num_train_epochs=120,
    per_device_train_batch_size=50,
    save_steps=10000,
    save_total_limit=2,
)

# Prepare the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

# Create a Trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model()


In [ ]:

user_prompt =  "How can I calculate my Grade Point Average?"

ft_model_name="./fine-tuned-model1"
ft_model = GPT2LMHeadModel.from_pretrained(ft_model_name).to('cuda:0')
# tokenizer = GPT2Tokenizer.from_pretrained(ft_model_name)
user_prompt_tokens = tokenizer.encode(user_prompt, return_tensors="pt").to('cuda:0')
attention_mask = torch.ones(user_prompt_tokens.shape, dtype=user_prompt_tokens.dtype, device=user_prompt_tokens.device)


response =model.generate(
    input_ids=user_prompt_tokens,
    max_length=50,  # Adjust the maximum response length as needed
    num_return_sequences=1,
    no_repeat_ngram_size=3,
    pad_token_id=tokenizer.eos_token_id,  # Set pad_token_id explicitly
    attention_mask=attention_mask
)

chatbot_response = tokenizer.decode(response[0], skip_special_tokens=True)
print("Chatbot:\n", chatbot_response)
